In [1]:
import pandas as pd
import numpy as np

In [ ]:
pieces = []
years = range(1880, 2011)
columns = ['name', 'sex', 'births']

for year in years:
    path = 'Names/yob%d.txt' % year
    frame = pd.read_csv(path, names = columns)
    frame['year'] = year
    pieces.append(frame)
    names = pd.concat(pieces, ignore_index=True)
    
print(names)

names.to_pickle('names.pkl')

In [2]:
names = pd.read_pickle('names.pkl')

In [4]:
def get_topX(group, X):
    return group.sort_values(by='births', ascending = False)[:X]


def get_topK(year, k):
    grouped = names.groupby(['year'])
    topk = grouped.apply(get_topX, k)
    
    ret_ind = np.where(topk['year'] == year)[0].tolist()

    return(topk.iloc[ret_ind])

def frequency(name, year):
    ret_ind1 = np.where(names['year'] == year)[0].tolist()
    tdf = names.iloc[ret_ind1]
    
    ret_ind2 = np.where(tdf['name'] == name)[0].tolist()
    ndf = tdf.iloc[ret_ind2]
    
    return ndf.loc[ndf['sex'] == 'M']['births'].sum(), ndf.loc[ndf['sex'] == 'F']['births'].sum()

def relative_frequency(name, year):
    ret_ind1 = np.where(names['year'] == year)[0].tolist()
    tdf = names.iloc[ret_ind1]
    
    ret_ind2 = np.where(tdf['name'] == name)[0].tolist()
    ndf = tdf.iloc[ret_ind2]
    
    return ndf['births'].sum() / tdf['births'].sum()

def freq_G(name, gender, year):
    ret_ind1 = np.where(names['year'] == year)[0].tolist()
    tdf = names.iloc[ret_ind1]
    
    ret_ind2 = np.where(tdf['name'] == name)[0].tolist()
    ndf = tdf.iloc[ret_ind2]
    
    ret_ind3 = np.where(ndf['sex'] == gender)[0].tolist()
    sdf = ndf.iloc[ret_ind3]
    
    return sdf['births'].sum()
    

y = 1976
k = 3
top = get_topK(y, k)
print('Top %d names in %d:' % (k, y)) 
print(top)

s = 'Adam'
f = frequency(s, y)
print('%d male and %d female named %s in %d' % (f[0], f[1], s, y))

rf = relative_frequency(s, y)

print('Relative frequency of %s in %d: %f' %(s, y, rf))

def get_swaps():
    a = np.empty(0)
    
    name2 = names['name']
    name2.drop_duplicates(inplace=True)
    n = name2.sort_values(ascending=True)
    
    for name in n:
        if(did_swap(name)):
            a = np.append(a, name)
            
    return a        
        
        
def did_swap(name):
    fmp = False
    mmp = False
    
    m1880 = freq_G(name, 'M', 1880)
    m2011 = freq_G(name, 'M', 2010)
    f1880 = freq_G(name, 'F', 1880)
    f2011 = freq_G(name, 'F', 2010)
    
    if(m1880 == 0 and f1880 == 0):
        return False
    if(m2011 == 0 and f2011 == 0):
        return False
    
    mmp1880 = False
    mmp2011 = False
    
    if(m1880 > f1880):
        mmp1880 = True
    if(m2011 > f2011):
        mmp2011 = True

    if(mmp1880 != mmp2011):
        return True
    else:
        return False

#swap = get_swaps()
#print('Gender Swaps:')
#print(swap)

Top 3 names in 1976:
                 name sex  births  year
year                                   
1976 801849   Michael   M   66966  1976
     790947  Jennifer   F   59478  1976
     801850     Jason   M   52687  1976
9946 male and 45 female named Adam in 1976
Relative frequency of Adam in 1976: 0.003292
